# Load data

In [ ]:
import skimage.io as io
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from skimage.segmentation import relabel_sequential

In [ ]:
image_vol = io.imread ("CREMI_dataset/image/test_volume.tif")
lbl_vol = io.imread ("CREMI_dataset/label/boundary_label.tif")

print(f"image_vol has shape: {image_vol.shape}")
print(f"lbl_vol has shape: {lbl_vol.shape}")

In [ ]:
plt.figure(figsize=(20,20))
ax1 = plt.subplot(1,2,1)
ax1.imshow(image_vol[0], cmap=plt.cm.gray)
ax2 = plt.subplot(1,2,2)
ax2.imshow(lbl_vol[0])
plt.show()

# Random crop data into a chosen size

**Input**

In [ ]:
# Choose crop size
#
crop_size = (448,448)    # (256,256) , (448,448), (640,640)

In [ ]:
# Split data to train and valid
# Choose the number of train set and valid set
#
train_len = 103   # 128 * 0.8 = 103
val_len = 25      # 128 * 0.2 = 25

In [ ]:
# Seed
#
seed = 100

**Run**

In [ ]:
if not isinstance(crop_size,(list, tuple)):
    raise ValueError("crop_size has to be tuble or list")

train_img = []
train_lbl = []

val_img = []
val_lbl = []

img_size = image_vol[0].shape

rng = np.random.RandomState(seed)

In [ ]:
# Process train set. Crop random patch from each slice.
#
for _ in range (train_len):
    index = rng.randint(100) # Cremi has 125 slice, train from [0, 100]
    x = rng.randint(img_size[0] - crop_size[0])
    y = rng.randint(img_size[1] - crop_size[1])
    
    train_img.append(image_vol[index][x:x+crop_size[0], y:y+crop_size[1]])
    train_lbl.append(relabel_sequential(lbl_vol[index][x:x+crop_size[0], y:y+crop_size[1]])[0])
    
train_image_vol = np.array(train_img, dtype=np.uint8)
train_lbl_vol = np.array(train_lbl, dtype=np.int32)


print(f"train_image_vol has shape: {train_image_vol.shape}")
print(f"train_lbl_vol has shape: {train_lbl_vol.shape}")

n_list = [len(np.unique(train_lbl[i])) for i in range(len(train_lbl))]
maxn = np.max(n_list)
minn = np.min(n_list)
mead = np.median(n_list)

print(f"\nMax id number in train_lbl_vol: {maxn}")
print(f"Min id number in train_lbl_vol: {minn}")
print(f"Median id number in train_lbl_vol: {mead}")

plt.figure(figsize=(20,20))
ax1 = plt.subplot(1,2,1)
ax1.imshow(train_image_vol[0], cmap=plt.cm.gray)
ax2 = plt.subplot(1,2,2)
ax2.imshow(train_lbl_vol[0])
plt.show()

In [ ]:
# Process train set. Crop random patch from each slice.
#
for i in range (train_len, train_len+val_len):
    index = rng.randint(100,125) # Cremi has 125 slice, val from [100, 125]
    x = rng.randint(img_size[0] - crop_size[0])
    y = rng.randint(img_size[1] - crop_size[1])
    
    val_img.append(image_vol[index][x:x+crop_size[0], y:y+crop_size[1]])
    val_lbl.append(relabel_sequential(lbl_vol[index][x:x+crop_size[0], y:y+crop_size[1]])[0])
    
val_image_vol = np.array(val_img, dtype=np.uint8)
val_lbl_vol = np.array(val_lbl, dtype=np.int32)


print(f"val_image_vol has shape: {val_image_vol.shape}")
print(f"val_lbl_vol has shape: {val_lbl_vol.shape}")

n_list = [len(np.unique(val_lbl[i])) for i in range(len(val_lbl))]
maxn = np.max(n_list)
minn = np.min(n_list)
mead = np.median(n_list)

print(f"\nMax id number in val_lbl_vol: {maxn}")
print(f"Min id number in val_lbl_vol: {minn}")
print(f"Median id number in val_lbl_vol: {mead}")

plt.figure(figsize=(20,20))
ax1 = plt.subplot(1,2,1)
ax1.imshow(val_image_vol[0], cmap=plt.cm.gray)
ax2 = plt.subplot(1,2,2)
ax2.imshow(val_lbl_vol[0])
plt.show()

# Generate data for E2E  
Copy `cvppp` and `cvppp_test` in `to_E2E` to `data/` in `E2E` repo

In [ ]:
print("Generate data for E2E")
save_E2E_path = Path(str(crop_size[0])) / "to_E2E"
save_E2E_path_1 = save_E2E_path / "cvppp" / "A1"
save_E2E_path_2 = save_E2E_path / "cvppp_test" / "A1"
save_E2E_path_1.mkdir(parents=True, exist_ok=True)
save_E2E_path_2.mkdir(parents=True, exist_ok=True)


imgs = np.concatenate([train_image_vol,val_image_vol],0)
lbls = np.concatenate([train_lbl_vol,val_lbl_vol],0)
print("imgs.shape: ", imgs.shape)
print("lbls.shape: ", lbls.shape)
assert len(imgs)==len(lbls)
print("The maximum number of cell:",np.max(lbls))
print("\n")

for i in range(imgs.shape[0]):
    # Image
    im = np.repeat(np.expand_dims(imgs[i],-1),4,-1)
    im[:,:,-1] = 255
    io.imsave(str(save_E2E_path_1 / f"plant{i:03d}_rgb.png"),im.astype(np.uint8))
    io.imsave(str(save_E2E_path_2 / f"plant{i:03d}_rgb.png"),im.astype(np.uint8))
    
    # Label
    lbl, _, _ = relabel_sequential(lbls[i])
    lbl = np.repeat(np.expand_dims(lbl,-1),4,-1)
    lbl[:,:,-1] = 255
    io.imsave(str(save_E2E_path_1 / f"plant{i:03d}_label.png"),lbl.astype(np.uint8))
    io.imsave(str(save_E2E_path_2 / f"plant{i:03d}_label.png"),lbl.astype(np.uint8))
                
print("Done")
img = io.imread(str(save_E2E_path_1 / "plant000_rgb.png"))
lbl = io.imread(str(save_E2E_path_1 / "plant000_label.png"))

plt.figure(figsize=(20,20))
ax1 = plt.subplot(1,2,1)
ax1.imshow(img, cmap=plt.cm.gray)
ax2 = plt.subplot(1,2,2)
ax2.imshow(lbl)
plt.show()

# Save output volumes (Option)

In [ ]:
save_vol_path = Path(str(crop_size[0])) / "output_volumne"
save_vol_path.mkdir(parents=True, exist_ok=True)

io.imsave(str(save_vol_path / f"train_image_{crop_size[0]}.tif"), train_image_vol.astype(np.uint8))
io.imsave(str(save_vol_path / f"train_lbl_{crop_size[0]}.tif"), train_image_vol.astype(np.int32))

io.imsave(str(save_vol_path / f"val_image_{crop_size[0]}.tif"), val_image_vol.astype(np.uint8))
io.imsave(str(save_vol_path / f"val_lbl_{crop_size[0]}.tif"), val_lbl_vol.astype(np.int32))

print("Done")
print(f"Created volume in: {save_vol_path}")

# Evaluation

## Function

In [ ]:
# Evaluation functions
print("Evaluation functions")
import cv2
import numpy as np

import colorsys
from pathlib import Path
import skimage.io as io
import matplotlib.pyplot as plt
import imageio as iio

from skimage.measure import label
from skimage.segmentation import relabel_sequential
from sklearn.metrics import adjusted_rand_score
from pprint import pprint

from cremi.evaluation.voi import voi
from cremi.evaluation.rand import adapted_rand as ARAND



def GetDices(inLabel,gtLabel):
    if (inLabel.shape!=gtLabel.shape):
        print('Shapes of label images not identical.')
        return 0, 0

    inLabels = np.unique(inLabel)
    maxInLabel = np.amax(inLabels)
    maxGtLabel = np.amax(gtLabel)

    if(len(inLabels)<=1): # trivial solution
        print('Only one label given, assuming all background.')
        return 0, 0

    # calculate Dice between all labels using 2d histogram
    xedges = np.linspace( - 0.5, maxInLabel+0.5,maxInLabel+2) # interval edges of histogram
    yedges = np.linspace( - 0.5, maxGtLabel+0.5,maxGtLabel+2) # interval edges of histogram

    # histograms
    H2D, xedges, yedges = np.histogram2d(inLabel.flatten(), gtLabel.flatten(), bins=(xedges, yedges))
    inH1D, edges = np.histogram(inLabel.flatten(), bins=xedges)
    gtH1D, edges = np.histogram(gtLabel.flatten(), bins=yedges)

    # reshape 1d histograms for broadcast
    inH1D = np.reshape(inH1D,[len(inH1D),1])
    gtH1D = np.reshape(gtH1D,[1,len(gtH1D)])

    # best Dice is (2*overlap(A,B)/(size(A)+size(B)))
    perCombinationDice = 2*H2D/(inH1D + gtH1D + 1e-16)
    sMax = np.amax(perCombinationDice[1:,1:],1)
    bestDice = np.mean(sMax)

    # FgBgDice
    Overlap = np.sum(H2D[1:,1:])
    inFG = np.sum(inH1D[1:])
    gtFG = np.sum(gtH1D[:,1:])
    if ((inFG + gtFG)>1e-16):
        FgBgDice = 2*Overlap/(inFG + gtFG)
    else:
        FgBgDice = 1 # gt is empty and in has it found correctly

    return bestDice, FgBgDice

def DiffFGLabels(inLabel,gtLabel):
    if (inLabel.shape!=gtLabel.shape):
        return -1

    inLabels = np.unique(inLabel)
    gtLabels = np.unique(gtLabel)
    maxInLabel = np.int(np.max(inLabels)) # maximum label value in inLabel
    minInLabel = np.int(np.min(inLabels)) # minimum label value in inLabel
    maxGtLabel = np.int(np.max(gtLabels)) # maximum label value in gtLabel
    minGtLabel = np.int(np.min(gtLabels)) # minimum label value in gtLabel

    return  (maxInLabel-minInLabel) - (maxGtLabel-minGtLabel)

def getIOU(inLabel,gtLabel):
    intersect_cnt = np.count_nonzero (inLabel & gtLabel)
    union_cnt = np.count_nonzero (inLabel | gtLabel)
    if (union_cnt == 0):
        return 0
    return intersect_cnt / union_cnt

def is_overlap(inLabel,gtLabel, thres=0.5):
    result = np.count_nonzero(inLabel & gtLabel)
    gt_numpix = np.count_nonzero(gtLabel)

    if result > gt_numpix*thres:
        return True
    else:
        return False

def kitti_metric(inLabel,gtLabel):
    if (np.max(inLabel) == 0) and (np.max(gtLabel) == 0):
        return 0, 0, 0, 0

    inLabels = np.unique(inLabel).tolist()
    if 0 in inLabels:
        inLabels.remove(0)

    gtLabels = np.unique(gtLabel).tolist()
    if 0 in gtLabels:
        gtLabels.remove(0)

    MUCov = 0
    MWCov = 0
    AvgFP = 0
    AvgFN = 0
    fp_list = []
    fn_list = []
    for r_lbl in inLabels:
        if r_lbl == 0:
            continue
        fp = 0
        for r_G in gtLabels:
            if (r_G == 0) or (r_G in fp_list):
                continue

            if is_overlap(inLabel==r_lbl, gtLabel==r_G):
                fp +=1
                fp_list.append(r_G)
                break

        if fp == 0:
            AvgFP +=1

    if len(inLabels) != 0:
        AvgFP = AvgFP / len(inLabels)
    else: # len (inLabels) == 0:
        # if len(gtLabels) == 0:
        #   AvgFP = 0
        # elif len(gtLabels) != 0:
        #   AvgFP = 1
        AvgFP = 1

    #-------------------------
    # Calculate FN
    # -------------------------
    for r_G in gtLabels:
        if r_G == 0:
            continue
        fn = 0
        for r_lbl in inLabels:
            if (r_lbl == 0) or (r_lbl in fn_list):
                continue

            if is_overlap(inLabel==r_lbl, gtLabel==r_G):
                fn +=1
                fn_list.append(r_lbl)
                break

        if fn == 0:
            AvgFN +=1

    if len (gtLabels) != 0:
        AvgFN = AvgFN / len (gtLabels)
    else: # len (gtLabels) == 0:
        # if len(inLabels) == 0:
        #   AvgFN = 0
        # elif len(gtLabels) != 0:
        #   AvgFN = 1
        AvgFN = 1

    # -------------------------
    # Calculate MUcov and MWCov
    # -------------------------
    for r_G in gtLabels:
        if r_G == 0:
            continue
        max_iou = 0
        for r_lbl in inLabels:
            if r_lbl == 0:
                continue
            current_iou = getIOU(inLabel==r_lbl, gtLabel==r_G)
            if current_iou > max_iou:
                max_iou = current_iou

        num_r_g = np.count_nonzero(gtLabel == r_G)
        MUCov += max_iou
        MWCov += max_iou * num_r_g


    if len (gtLabels) != 0:
        MUCov = MUCov / len (gtLabels)
    else:
        MUCov = 0

    # MUCov = MUCov / len(gtLabels)
    MWCov = MWCov/(np.count_nonzero(gtLabel > 0) + 1)

    return MWCov, MUCov, AvgFP, AvgFN

def adjusted_rand_index (gt_lbl, pred_lbl):
    gt_lbl = gt_lbl.flatten ()
    pred_lbl = pred_lbl.flatten ()
    return adjusted_rand_score (gt_lbl, pred_lbl)


def clean (lbl, minsize=10):
    sizes = np.bincount (lbl.ravel ())
    mask_sizes = sizes >= minsize
    mask_sizes [0] = 0
    lbl = lbl * mask_sizes [lbl]
    return lbl

def get_separate_labels(label_img):
    w = 32
    l32 = label_img.astype('int32')
    l32i = ((l32[:, :, 0] << 2 * w) + (l32[:, :, 1] << w) + l32[:, :, 2])
    return relabel_sequential(l32i)[0].astype('int32')

def print_scores(score_list, name_list, index):
    num_len = len(score_list)
    print(f"\n\nImage: {index+103}")
    for i in range(num_len):
        print(f"{name_list[i]}: {score_list[i][index]}")
        
def score2str(score_list, name_list, name_text="Image"):
    str_list=[]
    len_item=len(score_list[0])
    len_name=len(name_list)
    for i in range(len_item):
        for j in range(len_name):
            if j==0:
                str_list.append(f"{name_text}: {i+103}\n{name_list[j]}: {np.around(score_list[j][i], decimals=4)}\n")
            else:
                str_list[i] +=(f"{name_list[j]}: {np.around(score_list[j][i], decimals=4)}\n")
    return str_list


def find_min_size_list(input_img_list, thres_val = 100):
    num_len = len(input_img_list)
    list_thres = []
    for i in range(num_len):
        _, y = np.unique(input_img_list[i], return_counts=True)
        yy = np.sort(y[y<thres_val])
        if len(yy) == 1:
            list_thres.append(yy[0]) 
            continue
        elif len(yy) == 0:
            continue
        diff_array = np.diff(np.sort(yy))
        list_thres.append(yy[int(np.where(diff_array == diff_array.max())[0])+1])
    return np.array(list_thres)
  
def color_generator (N):
    HSV_tuples = [(x/N, 0.5, (x%8)*0.07 + 0.5) for x in range(N)] # h, s, v

    # colorsys.hsv_to_rgb(h, s, v) range [0.0...1.0]
    RGB_tuples = list (map(lambda x: colorsys.hsv_to_rgb(*x), HSV_tuples))

    COLOR_LIST = [(int (rgb[0] * 255), int (rgb[1] * 255), int (rgb[2] * 255)) for rgb in RGB_tuples]
    # shuffle (COLOR_LIST)
    COLOR_LIST [0] = (0, 0, 0)

    def index2rgb (index):
        return COLOR_LIST [index]
    def lbl2rgb (lbl):
        original_shape = np.squeeze (lbl).shape
        lbl = lbl.flatten ().tolist ()
        lbl = list (map (index2rgb, lbl))
        lbl = np.array (lbl).reshape (original_shape + (3,))
        return lbl
    return lbl2rgb


def shuffle_Maxlbl (lbl, err_ids=None, N=100):
    """
    shuffle all lbl_ids in lbl with available id range from []
    :param lbl: lbl had been label again. Ids range from [0, number of cell ]
    :param err_ids: given the error_ids list, return the error_ids after shuffled
    :return:
    lbl after shuffled
    error_ids after shuffled if err_ids is not None
    """
    shuf_map = list(range(N))
    if 0 in shuf_map:
        shuf_map.remove(0)

    np.random.shuffle(shuf_map)
    lbl_ids = np.unique (lbl)
    shuffled = np.zeros_like (lbl)

    if err_ids is not None:
        tmp = [x for x in err_ids]

    for x in lbl_ids:
        if x == 0:
            continue
        shuffled [lbl==x] = shuf_map [x-1]

        if (err_ids is not None and x in err_ids):
            tmp[err_ids.index(x)] = shuf_map[x-1]

    if err_ids is not None:
        return shuffled, tmp

    return shuffled

def render_color(lbl, max_lbl):
    return lbl2rgb(shuffle_Maxlbl(lbl, N=max_lbl))


# relabel
def clean_reindex (lbl, minsize=40):
    lbl = clean (lbl, minsize)
    ret = np.zeros (lbl.shape, dtype=np.int32)
    cur_max_val = 0
    val_list = np.unique (lbl)
    for val in val_list:
        if (val == 0):
            continue
        mask = (lbl == val)
        sub_lbl = label (mask, connectivity=1).astype (np.int32)
        sub_lbl = clean (sub_lbl, minsize)
        sub_lbl = label (sub_lbl, connectivity=1).astype (np.int32)

        sub_lbl += cur_max_val * (sub_lbl > 0)
        ret += sub_lbl
        cur_max_val = np.max (ret)
    return ret
  
    
def post_processing(lbl, minsize=40, to_size=None):
    img = clean_reindex(lbl, minsize)
    if to_size is None:
        return img.astype('int32')
    else:
        return cv2.resize(img, to_size, interpolation=cv2.INTER_NEAREST).astype('int32')


def deploy_evaluate (lbl, gt_lbl, minsize=40):
#     pred_lbl = clean_reindex (lbl, minsize)
    pred_lbl = lbl

    bestDice, FgBgDice = GetDices (pred_lbl, gt_lbl)

    diffFG = DiffFGLabels (pred_lbl, gt_lbl)

    MWCov, MUCov, AvgFP, AvgFN  = kitti_metric(pred_lbl, gt_lbl)

    rand_i = adjusted_rand_index(gt_lbl,pred_lbl)
    
    (voi_split, voi_merge) = voi(np.expand_dims(lbl,0), np.expand_dims(gt_lbl,0), ignore_groundtruth = [0])
    adapted_rand = ARAND(np.expand_dims(lbl,0), np.expand_dims(gt_lbl,0))

    return bestDice, FgBgDice, diffFG , MWCov, MUCov, AvgFP, AvgFN, rand_i, voi_split, voi_merge, adapted_rand


def eval_img_list(input_lbl_list, input_gt_list):
    bestDices = []
    FgBgDices = []
    diffFGs = []
    MUCovs = []
    MWCovs = []
    AvgFPs = []
    AvgFNs = []
    rand_is = [] 
    voi_splits = []
    voi_merges = []
    adapted_rands = []
#     minsize = find_min_size(input_lbl_list, thres_val = 800).min()
#     print(f"use minsize: {minsize}")
    for i in range(len(input_lbl_list)):
#         bestDice, FgBgDice, diffFG, MWCov, MUCov, AvgFP, AvgFN, rand_i, voi_split, voi_merge, adapted_rand = deploy_evaluate(input_lbl_list[i], input_gt_list[i], minsize)
        bestDice, FgBgDice, diffFG, MWCov, MUCov, AvgFP, AvgFN, rand_i, voi_split, voi_merge, adapted_rand = deploy_evaluate(input_lbl_list[i], input_gt_list[i])
        bestDices.append(bestDice)
        FgBgDices.append(FgBgDice)
        diffFGs.append(abs(diffFG))
        MWCovs.append(MWCov)
        MUCovs.append(MUCov)
        AvgFPs.append(AvgFP)
        AvgFNs.append(AvgFN)
        rand_is.append(rand_i)
        voi_splits.append(voi_split)
        voi_merges.append(voi_merge)
        adapted_rands.append(adapted_rand)

    log_info = {
        "bestDice": np.mean(bestDices),
        "FgBgDice": np.mean(FgBgDices),
        "diffFG": np.mean(diffFGs),
        "MWCov": np.mean(MWCovs),
        "MUCov": np.mean(MUCovs),
        "AvgFP": np.mean(AvgFPs),
        "AvgFN": np.mean(AvgFNs),
        "rand_i": np.mean(rand_is),
        "voi_split": np.mean(voi_splits),
        "voi_merge": np.mean(voi_merges),
        "adapted_RAND": np.mean(adapted_rands),
        }
    return log_info, [bestDices, FgBgDices, diffFGs, MWCovs, MUCovs, AvgFPs, AvgFNs, rand_is, voi_splits, voi_merges, adapted_rands]

## For CREMI 256 or 448

In [ ]:
image_size = 448 # 256 or 448
thres_val_pix = 300

if image_size==256:   
    test_raw_vol = iio.volread("256/test_raw.tif")
    test_lbl_vol = iio.volread("256/test_lbl.tif")
    E2E_result_path = Path("E2E_256_result/30")
    GrCL_result_vol = iio.volread("GraphCL_result/cremi/deploy_256_cremi.tif")

elif image_size==448:
    test_raw_vol = iio.volread("448/test_raw.tif")
    test_lbl_vol = iio.volread("448/test_lbl.tif")
    E2E_result_path = Path("E2E_448_fixed_order_result/30")
    GrCL_result_vol = iio.volread("GraphCL_result/cremi/deploy_448_cremi.tif")

print(f"Evaluate image size {image_size} with thres {thres_val_pix}")
    
n_max_lbl = np.max(test_lbl_vol) + 30
print("n_max_lbl: ", n_max_lbl)
lbl2rgb = color_generator(n_max_lbl)

E2E_result_vol=[get_separate_labels(io.imread(str(img_path))) for img_path in E2E_result_path.glob("*.png")]    
E2E_minsize=find_min_size_list(E2E_result_vol, thres_val = thres_val_pix).min()
GrCL_minsize=find_min_size_list(GrCL_result_vol, thres_val = thres_val_pix).min()
print(f"\nEval E2E: {E2E_minsize}")
print(f"\nEval GrCL: {GrCL_minsize}")

# For evaluation
E2E_list=[]
GrCL_list=[]
gt_lbl=[]

# For plot image
list_plt_imgs_name = []
list_plt_imgs = []

for i, img_path in enumerate(E2E_result_path.glob("*.png")):
    list_plt_imgs_name.append(f"test_raw_{i+103}")
    list_plt_imgs_name.append(f"test_lbl_{i+103}")
    list_plt_imgs_name.append(f"E2E_result_{i+103}")
    list_plt_imgs_name.append(f"GraphCL_result{i+103}")
    
    E2E_img = io.imread(str(img_path))
    GrCL_img = post_processing(GrCL_result_vol[i], minsize=GrCL_minsize, to_size=(image_size, image_size))

    list_plt_imgs.append(test_raw_vol[i])
    list_plt_imgs.append(render_color(test_lbl_vol[i], max_lbl=n_max_lbl))
    list_plt_imgs.append(E2E_img)
    list_plt_imgs.append(render_color(GrCL_img, max_lbl=n_max_lbl))
    
    gt_lbl.append(test_lbl_vol[i].astype('int32'))
    E2E_list.append(get_separate_labels(E2E_img))
#     E2E_list.append(post_processing(get_separate_labels(E2E_img), minsize=E2E_minsize, to_size=(image_size, image_size)))
    GrCL_list.append(GrCL_img)


#### ---------------- Evaluation ---------------- ####
E2E_log_info, E2E_score_list = eval_img_list(E2E_list, gt_lbl)
GrCL_log_info, GrCL_score_list = eval_img_list(GrCL_list, gt_lbl)

print("\nE2E_log_info:")
pprint(E2E_log_info)

print("\nGrCL_log_info:")
pprint(GrCL_log_info)

In [ ]:
## ---------------- print  ---------------- ####
score_list_name = ["bestDice", "FgBgDice", "diffFG", 
                   "MWCov", "MUCov", "AvgFP", "AvgFN", "rand_i", 
                   "voi_split", "voi_merge", "adapted_rand"]

E2E_str_score= score2str(E2E_score_list, score_list_name, "E2E")
GrCL_str_score= score2str(GrCL_score_list, score_list_name, "GrCL")

#### ---------------- Plot  ---------------- #### 
show_one = False
show_text = False

export_image = False
export_all = False

if image_size==448:
    export_list_num = [2,3,15,21] # 448
elif image_size==256:
    export_list_num = [0,1,3,4,5,6,9,10,12,15,16,18,20,21,22,24] # 256
else:
    export_list_num = [2,3,15,21]

def plt_hide_all(ax):
    ax.axis(False)
    # Hide grid lines
    ax.grid(False)
    # Hide axes ticks
    ax.set_xticks([])
    ax.set_yticks([])

def plot_im(imgs, col=4):
    num_img = len(imgs)
    if show_text:
        col += 1
        row = num_img//(col-1) + 1
    else:
        row = num_img//col + 1
    plt.figure(figsize=(20,150))
    for i in range(num_img):
        if export_image:         
            if (i//col) in export_list_num:
                iio.imwrite(f"paper_images/{list_plt_imgs_name[i]}.png",imgs[i])
            elif export_all:
                iio.imwrite(f"paper_images/{list_plt_imgs_name[i]}.png",imgs[i])
        if show_one and i == col:
            break
        if show_text:
            axi = plt.subplot(row, col, i + 1 + i//(col-1))
        else:
            axi = plt.subplot(row, col, i + 1)
        plt_hide_all(axi)
        axi.set_title(f"{i//col}-{list_plt_imgs_name[i]}")
#         axi.set_title(f"{i//col}")
        if "raw" in list_plt_imgs_name[i]:
            axi.imshow(imgs[i], cmap=plt.cm.gray)
        elif "lbl" in list_plt_imgs_name[i]:
            axi.imshow(imgs[i])
        elif "E2E" in list_plt_imgs_name[i]:
            axi.imshow(imgs[i])
        else:
            axi.imshow(imgs[i])
            if show_text:
                axtext = plt.subplot(row, col, i + 1 + i//(col-1) + 1)
                plt_hide_all(axtext)
                axtext.text(0, 0, E2E_str_score[i//(col-1)], fontsize=10)
                axtext.text(1.5, 0, GrCL_str_score[i//(col-1)], fontsize=10)       
    plt.show()
plot_im(list_plt_imgs)

## Only CREMI 640x640

In [ ]:
ver_deploy = 2 # 2 or 3
thres_val_pix = 100

image_size = 640 
test_raw_vol = iio.volread("640/test_raw.tif")
test_lbl_vol = iio.volread("640/test_lbl.tif")

if ver_deploy==2:
    GrCL_result_vol = iio.volread("GraphCL_result/cremi/deploy_640_ver2_cremi.tif")
elif ver_deploy==3:
    GrCL_result_vol = iio.volread("GraphCL_result/cremi/deploy_640_ver3_cremi.tif")

print(f"Evaluate image size {image_size} version {ver_deploy} with thres {thres_val_pix}")
    
n_max_lbl = np.max(test_lbl_vol) + 30
print("n_max_lbl: ", n_max_lbl)
lbl2rgb = color_generator(n_max_lbl)
    
GrCL_minsize=find_min_size_list(GrCL_result_vol, thres_val = thres_val_pix).min()
print(f"\nEval GrCL: {GrCL_minsize}")

# For evaluation
GrCL_list=[]
gt_lbl=[]

# For plot image
list_plt_imgs_name = []
list_plt_imgs = []

for i in range(len(GrCL_result_vol)):
    list_plt_imgs_name.append(f"test_raw_{i}")
    list_plt_imgs_name.append(f"test_lbl_{i}")
    list_plt_imgs_name.append(f"GraphCL_result{i}")
    
    GrCL_img = post_processing(GrCL_result_vol[i], minsize=GrCL_minsize, to_size=(image_size, image_size))

    list_plt_imgs.append(test_raw_vol[i])
    list_plt_imgs.append(render_color(test_lbl_vol[i], max_lbl=n_max_lbl))
    list_plt_imgs.append(render_color(GrCL_img, max_lbl=n_max_lbl))
    
    gt_lbl.append(test_lbl_vol[i].astype('int32'))
    GrCL_list.append(GrCL_img)

#### ---------------- Evaluation ---------------- ####
# GrCL_log_info, GrCL_score_list = eval_img_list(GrCL_list, gt_lbl)

# print("\nGrCL_log_info:")
# pprint(GrCL_log_info)

print("\nDone")

In [ ]:
## ---------------- print  ---------------- ####
score_list_name = ["bestDice", "FgBgDice", "diffFG", 
                   "MWCov", "MUCov", "AvgFP", "AvgFN", "rand_i", 
                   "voi_split", "voi_merge", "adapted_rand"]

GrCL_str_score= score2str(GrCL_score_list, score_list_name, "GrCL")

#### ---------------- Plot  ---------------- #### 
show_one = False
show_text = False

export_image = True
export_list_num = [0,2,3]

def plt_hide_all(ax):
    ax.axis(False)
    # Hide grid lines
    ax.grid(False)
    # Hide axes ticks
    ax.set_xticks([])
    ax.set_yticks([])

def plot_im(imgs, col=3):
    num_img = len(imgs)
    if show_text:
        col += 1
        row = num_img//(col-1) + 1
    else:
        row = num_img//col + 1
    plt.figure(figsize=(15,150))
    for i in range(num_img):
        if export_image:         
            if (i//col) in export_list_num:
                iio.imwrite(f"paper_images/{list_plt_imgs_name[i]}.png",imgs[i])
        if show_one and i == col:
            break
        if show_text:
            axi = plt.subplot(row, col, i + 1 + i//(col-1))
        else:
            axi = plt.subplot(row, col, i + 1)
        plt_hide_all(axi)
        axi.set_title(f"{i//col}-{list_plt_imgs_name[i]}")
#         axi.set_title(f"{i//col}")
        if "raw" in list_plt_imgs_name[i]:
            axi.imshow(imgs[i], cmap=plt.cm.gray)
        elif "lbl" in list_plt_imgs_name[i]:
            axi.imshow(imgs[i])
        elif "E2E" in list_plt_imgs_name[i]:
            axi.imshow(imgs[i])
        else:
            axi.imshow(imgs[i])
            if show_text:
                axtext = plt.subplot(row, col, i + 1 + i//(col-1) + 1)
                plt_hide_all(axtext)
                axtext.text(0, 0, E2E_str_score[i//(col-1)], fontsize=10)
                axtext.text(1.5, 0, GrCL_str_score[i//(col-1)], fontsize=10)       
    plt.show()
plot_im(list_plt_imgs)

# For Appendix & Draw Fig 1

## CVPPP result 

In [ ]:
print("cvppp result paper")

import colorsys
import cv2
import numpy as np

from pathlib import Path
import skimage.io as io
import matplotlib.pyplot as plt
import imageio as iio

from skimage.measure import label
from skimage.segmentation import relabel_sequential

def get_path(row, col, size, img):
    row_i = row + 1
    col_i = col + 1
    return img[size*row:size*row_i, size*col:size*col_i,:]

def im2pic(img):
    return np.repeat(np.expand_dims(img,-1),3,-1)

def color_generator (N):
    HSV_tuples = [(x/N, 0.5, (x%8)*0.07 + 0.5) for x in range(N)] # h, s, v

    # colorsys.hsv_to_rgb(h, s, v) range [0.0...1.0]
    RGB_tuples = list (map(lambda x: colorsys.hsv_to_rgb(*x), HSV_tuples))

    COLOR_LIST = [(int (rgb[0] * 255), int (rgb[1] * 255), int (rgb[2] * 255)) for rgb in RGB_tuples]
    # shuffle (COLOR_LIST)
    COLOR_LIST [0] = (0, 0, 0)

    def index2rgb (index):
        return COLOR_LIST [index]
    def lbl2rgb (lbl):
        original_shape = np.squeeze (lbl).shape
        lbl = lbl.flatten ().tolist ()
        lbl = list (map (index2rgb, lbl))
        lbl = np.array (lbl).reshape (original_shape + (3,))
        return lbl
    return lbl2rgb

def shuffle_Maxlbl (lbl, err_ids=None, N=100):
    """
    shuffle all lbl_ids in lbl with available id range from []
    :param lbl: lbl had been label again. Ids range from [0, number of cell ]
    :param err_ids: given the error_ids list, return the error_ids after shuffled
    :return:
    lbl after shuffled
    error_ids after shuffled if err_ids is not None
    """
    shuf_map = list(range(N))
    if 0 in shuf_map:
        shuf_map.remove(0)

    np.random.shuffle(shuf_map)
    lbl_ids = np.unique (lbl)
    shuffled = np.zeros_like (lbl)

    if err_ids is not None:
        tmp = [x for x in err_ids]

    for x in lbl_ids:
        if x == 0:
            continue
        shuffled [lbl==x] = shuf_map [x-1]

        if (err_ids is not None and x in err_ids):
            tmp[err_ids.index(x)] = shuf_map[x-1]

    if err_ids is not None:
        return shuffled, tmp

    return shuffled


lbl2rgb = color_generator(100)

lbl = io.imread("GraphCL_result/cvppp/1.png")
# name_deploy = "deploy_4paper_256_cremi_deploy_AttUNet2_masks_seg_256_cremi_39250.tif"
# num_deploy = 10

# lbl = io.imread(f"/home/Alexandrite/khoa/Segmentation_RL/CVPR/tuan_GraphCL/deploy/{name}")[num_deploy]

patch_size = 256

save_path = f"paper_images/for_fig_1/cremi/{patch_size}"

lbl_coor=(lbl.shape[0]//patch_size, lbl.shape[1]//patch_size)
print(lbl.shape, f"==> row={lbl_coor[0]}, col={lbl_coor[1]}")

train_lbl = iio.volread("/home/Alexandrite/tuan/GraphCL/Data/CVPPP_Challenge/Have_background/valid/A/valid_set_A.tif")
valid_lbl = iio.volread("/home/Alexandrite/tuan/GraphCL/Data/CVPPP_Challenge/Have_background/valid/B/valid_set_B.tif")

train_lbl_img= cv2.resize(train_lbl[1], (patch_size,patch_size), interpolation=cv2.INTER_NEAREST) 
valid_lbl_img= lbl2rgb(shuffle_Maxlbl((cv2.resize(valid_lbl[1], (patch_size,patch_size), interpolation=cv2.INTER_NEAREST))))

# iio.imwrite(f"{save_path}/train.png",train_lbl_img)
# iio.imwrite(f"{save_path}/valid.png",valid_lbl_img)

j=0
for i in range (11,0,-2):
    path_i = get_path(row=i, col=1, size=256, img=lbl)
#     iio.imwrite(f"{save_path}/lbl_result_{j:03d}.png",path_i)
    valid_lbl_img = np.hstack((valid_lbl_img,path_i))
    j+=1

j=0   
for i in range (3,9,1):
    path_i = get_path(row=1, col=i, size=256, img=lbl)
#     iio.imwrite(f"{save_path}/action_result_{j:03d}.png",path_i)
    train_lbl_img = np.hstack((train_lbl_img, path_i))
    j+=1
    
paper_img = np.vstack((train_lbl_img, valid_lbl_img))
    
iio.imwrite(f"{save_path}/paper_img.png",paper_img)

plt.imshow(paper_img)
plt.show()



In [ ]:
print("CVPPP result paper appedix run all test set")

import colorsys
import cv2
import numpy as np

from pathlib import Path
import skimage.io as io
import matplotlib.pyplot as plt
import imageio as iio

from skimage.measure import label
from skimage.segmentation import relabel_sequential

def get_path(row, col, size, img):
    # Use index 0
    row_i = row + 1
    col_i = col + 1
    return img[size*row:size*row_i, size*col:size*col_i,:]

def im2pic(img):
    return np.repeat(np.expand_dims(img,-1),3,-1)

def color_generator (N):
    HSV_tuples = [(x/N, 0.5, (x%8)*0.07 + 0.5) for x in range(N)] # h, s, v

    # colorsys.hsv_to_rgb(h, s, v) range [0.0...1.0]
    RGB_tuples = list (map(lambda x: colorsys.hsv_to_rgb(*x), HSV_tuples))

    COLOR_LIST = [(int (rgb[0] * 255), int (rgb[1] * 255), int (rgb[2] * 255)) for rgb in RGB_tuples]
    # shuffle (COLOR_LIST)
    COLOR_LIST [0] = (0, 0, 0)

    def index2rgb (index):
        return COLOR_LIST [index]
    def lbl2rgb (lbl):
        original_shape = np.squeeze (lbl).shape
        lbl = lbl.flatten ().tolist ()
        lbl = list (map (index2rgb, lbl))
        lbl = np.array (lbl).reshape (original_shape + (3,))
        return lbl
    return lbl2rgb

def shuffle_Maxlbl (lbl, err_ids=None, N=100):
    """
    shuffle all lbl_ids in lbl with available id range from []
    :param lbl: lbl had been label again. Ids range from [0, number of cell ]
    :param err_ids: given the error_ids list, return the error_ids after shuffled
    :return:
    lbl after shuffled
    error_ids after shuffled if err_ids is not None
    """
    shuf_map = list(range(N))
    if 0 in shuf_map:
        shuf_map.remove(0)

    np.random.shuffle(shuf_map)
    lbl_ids = np.unique (lbl)
    shuffled = np.zeros_like (lbl)

    if err_ids is not None:
        tmp = [x for x in err_ids]

    for x in lbl_ids:
        if x == 0:
            continue
        shuffled [lbl==x] = shuf_map [x-1]

        if (err_ids is not None and x in err_ids):
            tmp[err_ids.index(x)] = shuf_map[x-1]

    if err_ids is not None:
        return shuffled, tmp

    return shuffled


lbl2rgb = color_generator(100)
name_deploy = "deploy_4paper_cvppp_deploy_ruby_3_14112019_AttUNet2_masks_seg_cvppp_45420.tif"
data = "testset" # testset or validset
patch_size = 176
save_path = f"paper_images/for_fig_1/cvppp"

lbl = io.imread(f"/home/Alexandrite/khoa/Segmentation_RL/CVPR/GraphCL/deploy/{name_deploy}")
num_deploy = len(lbl)

lbl_coor=(lbl.shape[1]//patch_size, lbl.shape[2]//patch_size)
print(lbl.shape, f"==> row={lbl_coor[0]}, col={lbl_coor[1]}")

padding_array = im2pic(np.ones((patch_size,30))*255)

if data == "validset":
    train_lbl_vol = iio.volread("/home/Alexandrite/khoa/Segmentation_RL/CVPR/GraphCL/Data/CVPPP_Challenge/Have_background/valid/A/valid_set_A.tif")
    valid_lbl_vol = iio.volread("/home/Alexandrite/khoa/Segmentation_RL/CVPR/GraphCL/Data/CVPPP_Challenge/Have_background/valid/B/valid_set_B.tif")
elif data == "testset":
    train_lbl_vol = iio.volread("/home/Alexandrite/khoa/Segmentation_RL/CVPR/GraphCL/Data/CVPPP_Challenge/Have_background/test/A/test_set_A.tif")
    
if data == "testset":
    for i_img in range(num_deploy):
        if (i_img in [21,23,25,27,28,31]):
            continue
        train_lbl_img = cv2.resize(train_lbl_vol[i_img], (patch_size,patch_size), interpolation=cv2.INTER_NEAREST)
        
        train_lbl_img = np.hstack((train_lbl_img, padding_array))

        j=0
        for i in range (10,-1,-2):
            path_i = get_path(row=i, col=1, size=patch_size, img=lbl[i_img])
            train_lbl_img = np.hstack((train_lbl_img,path_i))
            j+=1

        paper_img = train_lbl_img
        iio.imwrite(f"{save_path}/paper_img_result_{i_img:03d}.png",paper_img)
        print(f"Done image {i_img}")
        
elif data == "validset":
    print("Chưa code cho valid set. Chừng nào cần thì code")
    

## CREMI result

In [ ]:
print("CREMI result paper appedix for 1 index")

import colorsys
import cv2
import numpy as np

from pathlib import Path
import skimage.io as io
import matplotlib.pyplot as plt
import imageio as iio

from skimage.measure import label
from skimage.segmentation import relabel_sequential

def get_path(row, col, size, img):
    # Use index 0
    row_i = row + 1
    col_i = col + 1
    return img[size*row:size*row_i, size*col:size*col_i,:]

def im2pic(img):
    return np.repeat(np.expand_dims(img,-1),3,-1)

def color_generator (N):
    HSV_tuples = [(x/N, 0.5, (x%8)*0.07 + 0.5) for x in range(N)] # h, s, v

    # colorsys.hsv_to_rgb(h, s, v) range [0.0...1.0]
    RGB_tuples = list (map(lambda x: colorsys.hsv_to_rgb(*x), HSV_tuples))

    COLOR_LIST = [(int (rgb[0] * 255), int (rgb[1] * 255), int (rgb[2] * 255)) for rgb in RGB_tuples]
    # shuffle (COLOR_LIST)
    COLOR_LIST [0] = (0, 0, 0)

    def index2rgb (index):
        return COLOR_LIST [index]
    def lbl2rgb (lbl):
        original_shape = np.squeeze (lbl).shape
        lbl = lbl.flatten ().tolist ()
        lbl = list (map (index2rgb, lbl))
        lbl = np.array (lbl).reshape (original_shape + (3,))
        return lbl
    return lbl2rgb

def shuffle_Maxlbl (lbl, err_ids=None, N=100):
    """
    shuffle all lbl_ids in lbl with available id range from []
    :param lbl: lbl had been label again. Ids range from [0, number of cell ]
    :param err_ids: given the error_ids list, return the error_ids after shuffled
    :return:
    lbl after shuffled
    error_ids after shuffled if err_ids is not None
    """
    shuf_map = list(range(N))
    if 0 in shuf_map:
        shuf_map.remove(0)

    np.random.shuffle(shuf_map)
    lbl_ids = np.unique (lbl)
    shuffled = np.zeros_like (lbl)

    if err_ids is not None:
        tmp = [x for x in err_ids]

    for x in lbl_ids:
        if x == 0:
            continue
        shuffled [lbl==x] = shuf_map [x-1]

        if (err_ids is not None and x in err_ids):
            tmp[err_ids.index(x)] = shuf_map[x-1]

    if err_ids is not None:
        return shuffled, tmp

    return shuffled


lbl2rgb = color_generator(100)
name_deploy = "deploy_4paper_256_cremi_deploy_AttUNet2_masks_seg_256_cremi_39250.tif"
num_deploy = 10
patch_size = 256

save_path = f"paper_images/for_fig_1/cremi/{patch_size}"

lbl = io.imread(f"/home/Alexandrite/khoa/Segmentation_RL/CVPR/tuan_GraphCL/deploy/{name_deploy}")[num_deploy]


lbl_coor=(lbl.shape[0]//patch_size, lbl.shape[1]//patch_size)
print(lbl.shape, f"==> row={lbl_coor[0]}, col={lbl_coor[1]}")

padding_array = im2pic(np.ones((patch_size,30))*255)

train_lbl = iio.volread(f"{patch_size}/test_raw.tif")[num_deploy]
valid_lbl = iio.volread(f"{patch_size}/test_lbl.tif")[num_deploy]

train_lbl_img = im2pic(cv2.resize(train_lbl, (patch_size,patch_size), interpolation=cv2.INTER_NEAREST)) 
valid_lbl_img = lbl2rgb(shuffle_Maxlbl((cv2.resize(valid_lbl, (patch_size,patch_size), interpolation=cv2.INTER_NEAREST))))

train_lbl_img = np.hstack((train_lbl_img,padding_array))
valid_lbl_img = np.hstack((valid_lbl_img, padding_array))

# iio.imwrite(f"{save_path}/train.png",train_lbl_img)
# iio.imwrite(f"{save_path}/valid.png",valid_lbl_img)

j=0
for i in range (10,-1,-2):
    path_i = get_path(row=i, col=1, size=256, img=lbl)
#     iio.imwrite(f"{save_path}/lbl_result_{j:00d}.png",path_i)
    valid_lbl_img = np.hstack((valid_lbl_img,path_i))
    j+=1


j=0
for i in range (3,9):
    path_i = get_path(row=0, col=i, size=256, img=lbl)
#     iio.imwrite(f"{save_path}/action_result_{j:03d}.png",path_i)
    train_lbl_img = np.hstack((train_lbl_img, path_i))
    j+=1
    
paper_img = np.vstack((train_lbl_img, valid_lbl_img))
    
iio.imwrite(f"{save_path}/paper_img.png",paper_img)

# plt.imshow(paper_img)
# plt.show()

In [ ]:
print("CREMI result paper appedix run all test set")

import colorsys
import cv2
import numpy as np

from pathlib import Path
import skimage.io as io
import matplotlib.pyplot as plt
import imageio as iio

from skimage.measure import label
from skimage.segmentation import relabel_sequential

def get_path(row, col, size, img):
    # Use index 0
    row_i = row + 1
    col_i = col + 1
    return img[size*row:size*row_i, size*col:size*col_i,:]

def im2pic(img):
    return np.repeat(np.expand_dims(img,-1),3,-1)

def color_generator (N):
    HSV_tuples = [(x/N, 0.5, (x%8)*0.07 + 0.5) for x in range(N)] # h, s, v

    # colorsys.hsv_to_rgb(h, s, v) range [0.0...1.0]
    RGB_tuples = list (map(lambda x: colorsys.hsv_to_rgb(*x), HSV_tuples))

    COLOR_LIST = [(int (rgb[0] * 255), int (rgb[1] * 255), int (rgb[2] * 255)) for rgb in RGB_tuples]
    # shuffle (COLOR_LIST)
    COLOR_LIST [0] = (0, 0, 0)

    def index2rgb (index):
        return COLOR_LIST [index]
    def lbl2rgb (lbl):
        original_shape = np.squeeze (lbl).shape
        lbl = lbl.flatten ().tolist ()
        lbl = list (map (index2rgb, lbl))
        lbl = np.array (lbl).reshape (original_shape + (3,))
        return lbl
    return lbl2rgb

def shuffle_Maxlbl (lbl, err_ids=None, N=100):
    """
    shuffle all lbl_ids in lbl with available id range from []
    :param lbl: lbl had been label again. Ids range from [0, number of cell ]
    :param err_ids: given the error_ids list, return the error_ids after shuffled
    :return:
    lbl after shuffled
    error_ids after shuffled if err_ids is not None
    """
    shuf_map = list(range(N))
    if 0 in shuf_map:
        shuf_map.remove(0)

    np.random.shuffle(shuf_map)
    lbl_ids = np.unique (lbl)
    shuffled = np.zeros_like (lbl)

    if err_ids is not None:
        tmp = [x for x in err_ids]

    for x in lbl_ids:
        if x == 0:
            continue
        shuffled [lbl==x] = shuf_map [x-1]

        if (err_ids is not None and x in err_ids):
            tmp[err_ids.index(x)] = shuf_map[x-1]

    if err_ids is not None:
        return shuffled, tmp

    return shuffled


data = 448 # 256 or 448 or 640


if data == 256:
    name_deploy = "deploy_4paper_256_cremi_deploy_AttUNet2_masks_seg_256_cremi_39250.tif"
    patch_size = 256
    save_path = f"paper_images/for_fig_1/cremi/256"
    lbl2rgb = color_generator(100)
elif data == 448:
    name_deploy = "deploy_4paper_448_cremi_deploy_AttUNet2_masks_seg_448_cremi_35800.tif"
    patch_size = 224
    save_path = f"paper_images/for_fig_1/cremi/448"
    lbl2rgb = color_generator(100)
elif data == 640:
    name_deploy = "deploy_4paper_640_ver2_cremi_AttUNet2_masks_seg_640_cremi_45650.tif"
    patch_size = 160
    save_path = f"paper_images/for_fig_1/cremi/640"
    lbl2rgb = color_generator(160)
else:
    print("What is this size???")

if data == 640:
    lbl = io.imread(f"/home/Alexandrite/khoa/Segmentation_RL/CVPR/GraphCL/deploy/{name_deploy}") # For 640 only
else:
    lbl = io.imread(f"/home/Alexandrite/khoa/Segmentation_RL/CVPR/tuan_GraphCL/deploy/{name_deploy}")

num_deploy = len(lbl)
print(f"Run CREMI full test set with size {patch_size}")

lbl_coor=(lbl.shape[1]//patch_size, lbl.shape[2]//patch_size)
print(lbl.shape, f"==> row={lbl_coor[0]}, col={lbl_coor[1]}")

padding_array = im2pic(np.ones((patch_size,30))*255)

train_lbl_vol = iio.volread(f"{data}/test_raw.tif")
valid_lbl_vol = iio.volread(f"{data}/test_lbl.tif")

for i_img in range(num_deploy):
    if (i_img in [0,3,4]) and data==256:
        continue
    if (i_img in [2,3,4,15]) and data==448:
        continue
    train_lbl_img = im2pic(cv2.resize(train_lbl_vol[i_img], (patch_size,patch_size), interpolation=cv2.INTER_NEAREST)) 
    valid_lbl_img = lbl2rgb(shuffle_Maxlbl((cv2.resize(valid_lbl_vol[i_img], (patch_size,patch_size), interpolation=cv2.INTER_NEAREST)), N=160 if data==640 else 100))
    
    train_lbl_img = np.hstack((train_lbl_img,padding_array))
    valid_lbl_img = np.hstack((padding_array,valid_lbl_img))
    
    j=0
    for i in range (10,-1,-2):
        path_i = get_path(row=i, col=1, size=patch_size, img=lbl[i_img])
        train_lbl_img = np.hstack((train_lbl_img,path_i))
        j+=1
    
    train_lbl_img = np.hstack((train_lbl_img,valid_lbl_img))
    
#     j=0
#     for i in range (10,-1,-2):
#         path_i = get_path(row=i, col=1, size=patch_size, img=lbl[i_img])
#         valid_lbl_img = np.hstack((valid_lbl_img,path_i))
#         j+=1
        
#     j=0
#     for i in range (3,9):
#         path_i = get_path(row=0, col=i, size=patch_size, img=lbl[i_img])
#         train_lbl_img = np.hstack((train_lbl_img, path_i))
#         j+=1

#     paper_img = np.vstack((train_lbl_img, valid_lbl_img))
    paper_img = train_lbl_img
    iio.imwrite(f"{save_path}/paper_img_result_{i_img:03d}.png",paper_img)
    print(f"Done image {i_img}")

## KITTI result

In [ ]:
print("KITTI result paper appedix run all test set")

import colorsys
import cv2
import numpy as np

from pathlib import Path
import skimage.io as io
import matplotlib.pyplot as plt
import imageio as iio

from skimage.measure import label
from skimage.segmentation import relabel_sequential

def get_path(row, col, size, img):
    # Use index 0
    row_i = row + 1
    col_i = col + 1
    return img[size[0]*row:size[0]*row_i, size[1]*col:size[1]*col_i,:]

def im2pic(img):
    return np.repeat(np.expand_dims(img,-1),3,-1)

def color_generator (N):
    HSV_tuples = [(x/N, 0.5, (x%8)*0.07 + 0.5) for x in range(N)] # h, s, v

    # colorsys.hsv_to_rgb(h, s, v) range [0.0...1.0]
    RGB_tuples = list (map(lambda x: colorsys.hsv_to_rgb(*x), HSV_tuples))

    COLOR_LIST = [(int (rgb[0] * 255), int (rgb[1] * 255), int (rgb[2] * 255)) for rgb in RGB_tuples]
    # shuffle (COLOR_LIST)
    COLOR_LIST [0] = (0, 0, 0)

    def index2rgb (index):
        return COLOR_LIST [index]
    def lbl2rgb (lbl):
        original_shape = np.squeeze (lbl).shape
        lbl = lbl.flatten ().tolist ()
        lbl = list (map (index2rgb, lbl))
        lbl = np.array (lbl).reshape (original_shape + (3,))
        return lbl
    return lbl2rgb

def shuffle_Maxlbl (lbl, err_ids=None, N=100):
    """
    shuffle all lbl_ids in lbl with available id range from []
    :param lbl: lbl had been label again. Ids range from [0, number of cell ]
    :param err_ids: given the error_ids list, return the error_ids after shuffled
    :return:
    lbl after shuffled
    error_ids after shuffled if err_ids is not None
    """
    shuf_map = list(range(N))
    if 0 in shuf_map:
        shuf_map.remove(0)

    np.random.shuffle(shuf_map)
    lbl_ids = np.unique (lbl)
    shuffled = np.zeros_like (lbl)

    if err_ids is not None:
        tmp = [x for x in err_ids]

    for x in lbl_ids:
        if x == 0:
            continue
        shuffled [lbl==x] = shuf_map [x-1]

        if (err_ids is not None and x in err_ids):
            tmp[err_ids.index(x)] = shuf_map[x-1]

    if err_ids is not None:
        return shuffled, tmp

    return shuffled


lbl2rgb = color_generator(100)
name_deploy = "deploy_4paper_kitti_deploy_1_14112019_AttUNet2_masks_seg_kitti_10200.tif"
patch_size = (160,480)
original_size = (1224, 370)

save_path = f"paper_images/for_fig_1/kitti"
lbl = io.imread(f"/home/Alexandrite/khoa/Segmentation_RL/CVPR/GraphCL/deploy/{name_deploy}")
num_deploy = len(lbl)

lbl_coor=(lbl.shape[1]//patch_size[0], lbl.shape[2]//patch_size[1])
print(lbl.shape, f"==> row={lbl_coor[0]}, col={lbl_coor[1]}")

padding_array = im2pic(np.ones((patch_size[0],30))*255)

train_lbl_vol = iio.volread("/home/Alexandrite/khoa/Segmentation_RL/CVPR/GraphCL/Data/kitti/valid/A/A_valid2.tif")
valid_lbl_vol = iio.volread("/home/Alexandrite/khoa/Segmentation_RL/CVPR/GraphCL/Data/kitti/valid/B/B_valid2.tif")
 
for i_img in range(num_deploy):
    object_in_img = len(np.unique(valid_lbl_vol[i_img]).tolist()) -1
    if object_in_img < 6 or i_img in [10, 6, 47, 62, 78, 96, 107, 108, 110]:
        continue
    train_lbl_img = cv2.resize(train_lbl_vol[i_img], (patch_size[1],patch_size[0]), interpolation=cv2.INTER_NEAREST)
    valid_lbl_img = lbl2rgb(shuffle_Maxlbl(cv2.resize(valid_lbl_vol[i_img], (patch_size[1],patch_size[0]), interpolation=cv2.INTER_NEAREST)))
    
    train_lbl_img = np.hstack((train_lbl_img,padding_array))
    valid_lbl_img = np.hstack((padding_array, valid_lbl_img))
    
    j=0
    for i in range (6,-1,-2):
        path_i = get_path(row=i, col=1, size=patch_size, img=lbl[i_img])
        train_lbl_img = np.hstack((train_lbl_img,path_i))
        j+=1
        
    train_lbl_img = np.hstack((train_lbl_img,valid_lbl_img))
        
    
#     j=0
#     for i in range (6,-1,-2):
#         path_i = get_path(row=i, col=1, size=patch_size, img=lbl[i_img])
#         valid_lbl_img = np.hstack((valid_lbl_img,path_i))
#         j+=1
        
#     j=0
#     for i in range (3,7):
#         path_i = get_path(row=0, col=i, size=patch_size, img=lbl[i_img])
#         train_lbl_img = np.hstack((train_lbl_img, path_i))
#         j+=1

#     paper_img = np.vstack((train_lbl_img, valid_lbl_img))
    paper_img = train_lbl_img
    iio.imwrite(f"{save_path}/paper_img_result_{i_img:03d}.png",paper_img)
    print(f"Done image {i_img}")

In [ ]:
print("Count how many object in Volume")
from collections import Counter
valid_lbl_vol = iio.volread("/home/Alexandrite/khoa/Segmentation_RL/CVPR/GraphCL/Data/kitti/valid/B/B_valid2.tif")
max_object = np.unique(valid_lbl_vol).max()
list_object = []
for i in range(len(valid_lbl_vol)):
    object_in_img = len(np.unique(valid_lbl_vol[i]).tolist()) -1
    list_object.append(object_in_img)

result_count= Counter(list_object)
print(result_count)

In [ ]:
print("Import Lib for preparing overlay raw and result")
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

import numpy as np
import cv2

import skimage.io as io
from skimage.measure import label, regionprops, find_contours
from skimage.color import gray2rgb

import random
import colorsys

import imageio
from io import BytesIO 

def clean (lbl, minsize=50):
    sizes = np.bincount (lbl.ravel ())
    mask_sizes = sizes >= minsize
    mask_sizes [0] = 0
    lbl = lbl * mask_sizes [lbl]
    return lbl

def relabel (lbl):
    lbl = clean (lbl)
    ret = np.zeros (lbl.shape, dtype=np.int32)
    cur_max_val = 0
    val_list = np.unique (lbl)
    for val in val_list:
        if (val == 0):
            continue
        mask = (lbl == val)
        sub_lbl = label (mask, connectivity=1).astype (np.int32)
        sub_lbl = clean (sub_lbl)
        sub_lbl = label (sub_lbl, connectivity=1).astype (np.int32)

        sub_lbl += cur_max_val * (sub_lbl > 0)
        ret += sub_lbl
        cur_max_val = np.max (ret)
    return ret

def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors


def apply_mask(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c] * 255,
                                  image[:, :, c])
    return image


def display_instances(image, masks,
                      figsize=(20, 20), 
                      title="", 
                      captions=False,
                      show_mask=True,
                      auto_show = False,
                      colors=None,
                      dpi=180):
    
    # Number of instances
    lbs_list = np.unique(masks).tolist()
    if 0 in lbs_list:
        lbs_list.remove(0)
    
    N = len(lbs_list)
    
    if not N:
        print("\n*** No instances to display *** \n")
        
#     _, ax = plt.subplots(1, figsize=figsize)
    fig, ax = plt.subplots(1, figsize=figsize)
    
    # Generate random colors
    colors = colors or random_colors(N)
    
    # Show area outside image boundaries.
    height, width = image.shape[:2]
#     ax.set_ylim(height + 10, -10)
#     ax.set_xlim(-10, width + 10)
    ax.axis('off')
    ax.set_title(title)
    
    masked_image = image.astype(np.uint32).copy()
    
    regions = regionprops(masks)
    
    for i in range(N):
        color = colors[i]

        # Label
        if captions:
            label = lbs_list[i]
            caption = "car {}".format(label)
            y1, x1, y2, x2 = regions[i].bbox
            ax.text(x1, y1 + 8, caption,
                    color='w', size=11, backgroundcolor="none")

        # Mask
        mask = masks==lbs_list[i]
        if show_mask:
            masked_image = apply_mask(masked_image, mask, color)

        # Mask Polygon
        # Pad to ensure proper polygons for masks that touch image edges.
        padded_mask = np.zeros(
            (mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = find_contours(padded_mask, 0.5)
        for verts in contours:
            # Subtract the padding and flip (y, x) to (x, y)
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", edgecolor=color)
            ax.add_patch(p)
    
# #     result_img = get_img_from_fig(fig, dpi)
#     fig.canvas.draw()
# #     data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
#     data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
# #     data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
#     data = data.reshape((1224, 370) + (3,))

    if auto_show:
        ax.imshow(masked_image.astype(np.uint8))
        
#         result_img = get_img_from_fig(fig, dpi)
        plt.show()
    return masked_image


def get_img_from_fig(fig, dpi=180):
    buf = BytesIO()
    fig.savefig(buf, format="png", dpi=180)
    buf.seek(0)
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    
    buf.close()
#     img_arr = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)

#     img_arr = img_arr.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    img_arr = img_arr.reshape((1224, 370) + (3,))
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    return img

In [ ]:
name_deploy = "deploy_kitti_deploy_1_14112019_AttUNet2_masks_seg_kitti_10200.tif"
patch_size = (160,480)

save_path = "paper_images/for_fig_1/kitti"

lbl = io.imread(f"/home/Alexandrite/khoa/Segmentation_RL/CVPR/GraphCL/deploy/{name_deploy}")
# raw_valid = io.imread ("../Data/kitti/test/A/A_test2.tif")
raw_test_vol = io.imread ("/home/Alexandrite/khoa/Segmentation_RL/CVPR/GraphCL/Data/kitti/valid/A/A_valid2.tif")
lbl_test_vol = iio.volread("/home/Alexandrite/khoa/Segmentation_RL/CVPR/GraphCL/Data/kitti/valid/B/B_valid2.tif")
# raw = io.imread ("../Data/kitti/train2/A/A_train2.tif")

# size_raw = len(raw)
# raw = [ raw[i] for i in range(700, size_raw) ]

# raw = np.concatenate([raw, raw_valid, raw_test], axis=0)
# raw = np.concatenate([raw_valid, raw_test], axis=0)
# raw = np.concatenate([raw_test, raw_valid], axis=0)

raw = raw_test_vol

original_size = (1224, 370) #WH

ret = []

results = np.zeros_like(raw)
results_val = np.zeros_like(raw)

# post processing
for img in lbl:
    img = relabel (img)
    ret.append (cv2.resize (img, original_size, interpolation=cv2.INTER_NEAREST))

ret = np.array (ret)
ret = ret.astype (np.uint8)

print("\nTotal images: ", ret.shape[0],"\n")

In [ ]:
num_list = []
for i in range (ret.shape[0]):
    show_img = True
#     show_img = False
#     object_in_img = len (np.unique (ret [idx])) -1
#     if object_in_img > 3:
#         print(f"Image {i}: {object_in_img} object")
#         num_list.append(i)
#         show_img = True
    
    results[i] = display_instances(raw [i],ret [i],
                                  auto_show = show_img)
    results_val[i] = display_instances(raw [i],lbl_test_vol [i],
                                  auto_show = False)
    

# imageio.volwrite("results_predict.tif",results.astype(np.int32))

In [ ]:
print(results[96].shape)
plt.imshow(results[96])
plt.show()

In [ ]:
print("KITTI result paper appedix run all test set")

import colorsys
import cv2
import numpy as np

from pathlib import Path
import skimage.io as io
import matplotlib.pyplot as plt
import imageio as iio

from skimage.measure import label
from skimage.segmentation import relabel_sequential

def get_path(row, col, size, img):
    # Use index 0
    row_i = row + 1
    col_i = col + 1
    return img[size[0]*row:size[0]*row_i, size[1]*col:size[1]*col_i,:]

def im2pic(img):
    return np.repeat(np.expand_dims(img,-1),3,-1)

def color_generator (N):
    HSV_tuples = [(x/N, 0.5, (x%8)*0.07 + 0.5) for x in range(N)] # h, s, v

    # colorsys.hsv_to_rgb(h, s, v) range [0.0...1.0]
    RGB_tuples = list (map(lambda x: colorsys.hsv_to_rgb(*x), HSV_tuples))

    COLOR_LIST = [(int (rgb[0] * 255), int (rgb[1] * 255), int (rgb[2] * 255)) for rgb in RGB_tuples]
    # shuffle (COLOR_LIST)
    COLOR_LIST [0] = (0, 0, 0)

    def index2rgb (index):
        return COLOR_LIST [index]
    def lbl2rgb (lbl):
        original_shape = np.squeeze (lbl).shape
        lbl = lbl.flatten ().tolist ()
        lbl = list (map (index2rgb, lbl))
        lbl = np.array (lbl).reshape (original_shape + (3,))
        return lbl
    return lbl2rgb

def shuffle_Maxlbl (lbl, err_ids=None, N=100):
    """
    shuffle all lbl_ids in lbl with available id range from []
    :param lbl: lbl had been label again. Ids range from [0, number of cell ]
    :param err_ids: given the error_ids list, return the error_ids after shuffled
    :return:
    lbl after shuffled
    error_ids after shuffled if err_ids is not None
    """
    shuf_map = list(range(N))
    if 0 in shuf_map:
        shuf_map.remove(0)

    np.random.shuffle(shuf_map)
    lbl_ids = np.unique (lbl)
    shuffled = np.zeros_like (lbl)

    if err_ids is not None:
        tmp = [x for x in err_ids]

    for x in lbl_ids:
        if x == 0:
            continue
        shuffled [lbl==x] = shuf_map [x-1]

        if (err_ids is not None and x in err_ids):
            tmp[err_ids.index(x)] = shuf_map[x-1]

    if err_ids is not None:
        return shuffled, tmp

    return shuffled


lbl2rgb = color_generator(100)
name_deploy = "deploy_4paper_kitti_deploy_1_14112019_AttUNet2_masks_seg_kitti_10200.tif"
patch_size = (160,480)
original_size = (370,1224)

upscale_ratio = 2.5
upscale_ratio_original_size = (int(original_size[0]*upscale_ratio), int(original_size[1]*upscale_ratio))

downscale_ratio = 4
downscale_ratio_original_size = (int(upscale_ratio_original_size[0]/downscale_ratio), int(upscale_ratio_original_size[1]/downscale_ratio))

save_path = f"paper_images/for_fig_1/kitti"
lbl = io.imread(f"/home/Alexandrite/khoa/Segmentation_RL/CVPR/GraphCL/deploy/{name_deploy}")
num_deploy = len(lbl)

lbl_coor=(lbl.shape[1]//patch_size[0], lbl.shape[2]//patch_size[1])
print(lbl.shape, f"==> row={lbl_coor[0]}, col={lbl_coor[1]}")

padding_array = im2pic(np.ones((30,original_size[1]))*255)
padding_array2 = im2pic(np.ones((30,upscale_ratio_original_size[1]))*255)


train_lbl_vol = iio.volread("/home/Alexandrite/khoa/Segmentation_RL/CVPR/GraphCL/Data/kitti/valid/A/A_valid2.tif")
valid_lbl_vol = iio.volread("/home/Alexandrite/khoa/Segmentation_RL/CVPR/GraphCL/Data/kitti/valid/B/B_valid2.tif")
 
for i_img in range(num_deploy):
    object_in_img = len(np.unique(valid_lbl_vol[i_img]).tolist()) -1
    if object_in_img < 6 or i_img in [6, 8, 10, 13, 14, 33, 47, 62, 78, 83, 96, 107, 108, 110]:
        continue
        
    train_img = train_lbl_vol[i_img]
    valid_lbl_img = lbl2rgb(shuffle_Maxlbl(valid_lbl_vol[i_img]))
    valid_lbl_img = np.vstack((padding_array, valid_lbl_img))
    
#     overlay_result = cv2.resize(results[i_img], (upscale_ratio_original_size[1], upscale_ratio_original_size[0]), interpolation=cv2.INTER_NEAREST)
#     overlay_valid = cv2.resize(results_val[i_img], (upscale_ratio_original_size[1], upscale_ratio_original_size[0]), interpolation=cv2.INTER_NEAREST)
#     big_overlay = np.vstack((overlay_result, padding_array2, overlay_valid))
    
    path_i_list = []
    j=0
    for i in range (6,-1,-2):
        path_i = cv2.resize(get_path(row=i, col=1, size=patch_size, img=lbl[i_img]), (original_size[1],original_size[0]), interpolation=cv2.INTER_NEAREST)
        path_i_list.append(path_i)
        j+=1
    
    path_i = np.vstack(path_i_list)
    valid_lbl_img = np.vstack((train_img, path_i, valid_lbl_img))
    
#     paper_img = cv2.resize(np.hstack((valid_lbl_img, big_overlay)), (downscale_ratio_original_size[1], downscale_ratio_original_size[0]), interpolation=cv2.INTER_NEAREST) 
    paper_img = valid_lbl_img
    
    iio.imwrite(f"{save_path}/paper_img_result_{i_img:03d}.png",paper_img)
    print(f"Done image {i_img}")